In [1]:
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model, Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPool2D, BatchNormalization, Dropout, Layer, LSTM, Input

In [2]:
dhw_merge = pd.read_csv('dhw_merge.csv')
elec_merge  = pd.read_csv('elec_merge.csv')
n_elec_merge = pd.read_csv('n_elec_merge.csv')

dhw_merge = dhw_merge.drop(['YEAR'], axis=1)
elec_merge = elec_merge.drop(['YEAR'], axis=1)
n_elec_merge = n_elec_merge.drop(['YEAR'], axis=1)

##  For Scaling

In [3]:
dhw_X = dhw_merge.drop(['DHW'], axis=1)
dhw_Y = dhw_merge[['DHW']]

scaler_dhw = MinMaxScaler()
scaler_dhw.fit(dhw_X)
scaled_dhw_X = scaler_dhw.transform(dhw_X)

new_dhw_X = pd.DataFrame(scaled_dhw_X, index=dhw_X.index, columns=dhw_X.columns)
new_dhw = pd.concat([new_dhw_X, dhw_Y], axis=1)

In [4]:
elec_X = elec_merge.drop(['ELEC'], axis=1)
elec_Y = elec_merge[['ELEC']]

scaler_elec = MinMaxScaler()
scaler_elec.fit(elec_X)
scaled_elec_X = scaler_elec.transform(elec_X)

new_elec_X = pd.DataFrame(scaled_elec_X, index=elec_X.index, columns=elec_X.columns)
new_elec = pd.concat([new_elec_X, elec_Y], axis=1)

In [5]:
n_elec_X = n_elec_merge.drop(['n_elec'], axis=1)
n_elec_Y = n_elec_merge[['n_elec']]

scaler_n_elec = MinMaxScaler()
scaler_n_elec.fit(n_elec_X)
scaled_n_elec_X = scaler_n_elec.transform(n_elec_X)

new_n_elec_X = pd.DataFrame(scaled_n_elec_X, index=n_elec_X.index, columns=n_elec_X.columns)
new_n_elec = pd.concat([new_n_elec_X, n_elec_Y], axis=1)

In [6]:
input_dhw = new_dhw.iloc[:-1,]
target_dhw = new_dhw[['DHW']].iloc[1:]

trainX_dhw, testX_dhw, trainY_dhw, testY_dhw = train_test_split(input_dhw,target_dhw,test_size=0.3,shuffle=False,random_state=0)

In [7]:
input_elec = new_elec.iloc[:-1,]
target_elec = new_elec[['ELEC']].iloc[1:]

trainX_elec, testX_elec, trainY_elec, testY_elec = train_test_split(input_elec,target_elec,test_size=0.3,shuffle=False,random_state=0)

In [8]:
input = new_n_elec.iloc[:-1,]
target = new_n_elec[['n_elec']].iloc[1:]

trainX_n_elec, testX_n_elec, trainY_n_elec, testY_n_elec = train_test_split(input,target,test_size=0.3,shuffle=False,random_state=0)

In [9]:
def buildDataSet(input, target, seqLength):
    xdata = []
    ydata = []
    for i in range(len(input) - seqLength):
        tx = input.iloc[i:i+seqLength]
        ty = target.iloc[i+seqLength-1]
        xdata.append(tx)
        ydata.append(ty)
    return np.array(xdata), np.array(ydata)

In [10]:
trainx_dhw, trainy_dhw = buildDataSet(trainX_dhw, trainY_dhw, 3)
testx_dhw, testy_dhw = buildDataSet(testX_dhw, testY_dhw, 3)
trainx_elec, trainy_elec = buildDataSet(trainX_elec, trainY_elec, 3)
testx_elec, testy_elec = buildDataSet(testX_elec, testY_elec, 3)
trainx_n_elec, trainy_n_elec = buildDataSet(trainX_n_elec, trainY_n_elec, 3)
testx_n_elec, testy_n_elec = buildDataSet(testX_n_elec, testY_n_elec, 3)

## Reverse MinMaxScaler

In [11]:
# reverse_train_x = scaler_dhw_x_train.inverse_transform(scaled_dhw_train_x)
# reverse_train_y = scaler_dhw_y_train.inverse_transform(scaled_dhw_train_y)
# reverse_test_x = scaler_dhw_x_train.inverse_transform(scaled_dhw_test_x)
# reverse_test_y = scaler_dhw_y_train.inverse_transform(scaled_dhw_test_y)

# rev_dhw_train_x = pd.DataFrame(reverse_train_x, index=dhw_x_train.index, columns=dhw_x_train.columns)
# rev_dhw_train_y = pd.DataFrame(reverse_train_y, index=dhw_y_train.index, columns=dhw_y_train.columns)

# rev_dhw_test_x = pd.DataFrame(reverse_test_x, index=dhw_x_test.index, columns=dhw_x_test.columns)
# rev_dhw_test_y = pd.DataFrame(reverse_test_y, index=dhw_y_test.index, columns=dhw_y_test.columns)

# rev_dhw_trainSet = pd.concat([rev_dhw_train_x, rev_dhw_train_y], axis=1)
# rev_dhw_testSet = pd.concat([rev_dhw_test_x, rev_dhw_test_y], axis=1)

In [12]:
class DANN(Model):
    def __init__(self):
        super(DANN, self).__init__()

        # Feature Extractor
        self.feature_extractor = Sequential([
           LSTM(64, activation='swish', return_sequences=True),
           LSTM(64, activation='swish', return_sequences=False)
        ])
        
        # Label Predictor
        self.label_predictor = Sequential([
            Flatten(),
            Dense(32, activation='relu'),
            Dense(1) # 에너지 소비량을 예측하기 위한 레이어
        ])

        self.predict_label = Sequential([
			self.feature_extractor,
			self.label_predictor
		])
        
    def call(self, inputs):
        prediction = self.predict_label(inputs)
        #prediction = self.label_predictor(lstm_features)
        
        return prediction

In [13]:
dann_model = DANN()

In [14]:
# Define your optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Define the loss function
loss_object = tf.keras.losses.MeanSquaredError()

# Training loop
num_epochs = 100
batch_size = 1024

In [19]:
# EarlyStopping 콜백을 생성합니다.
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='loss',      # 모니터링할 성능 지표 (여기서는 검증 손실)
    patience=25,              # 성능이 향상되지 않더라도 몇 번까지 기다릴지 지정
    mode='min',              # 성능 지표가 최소화되어야 하는지 또는 최대화되어야 하는지 지정
    verbose=0                # 조기 종료가 적용될 때 메시지를 출력 (1: 출력, 0: 출력하지 않음)
)

# ModelCheckpoint 콜백을 생성합니다.
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='model_weights.h5',  # 가중치 파일을 저장할 경로와 이름 지정
    save_best_only=True,          # 가장 좋은 성능을 보인 에포크의 가중치만 저장
    save_weights_only=True,       # 가중치만 저장 (아키텍처는 저장하지 않음)
    monitor='loss',           # 모니터링할 성능 지표 (여기서는 검증 손실)
    mode='min',                   # 성능 지표가 최소화되어야 하는지 또는 최대화되어야 하는지 지정
    verbose=0                     # 저장되는 과정을 출력 (1: 출력, 0: 출력하지 않음)
)

# EarlyStopping 콜백의 멤버 변수를 초기화합니다.
early_stopping_callback.stopped_epoch = None

for epoch in range(num_epochs):
    for i in range(0, len(trainx_n_elec), batch_size):
        batch_x = trainx_n_elec[i:i+batch_size]
        batch_y = trainy_n_elec[i:i+batch_size]
        
        with tf.GradientTape() as Tape:
            predictions = dann_model(batch_x)
            loss = loss_object(batch_y, predictions)
            
        gradients = Tape.gradient(loss, dann_model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, dann_model.trainable_variables))
        
        
    train_loss = loss_object(trainy_n_elec, dann_model(trainx_n_elec))
 
    # EarlyStopping과 ModelCheckpoint 콜백 적용
    early_stopping_callback.on_epoch_end(epoch, {'loss': train_loss})
    checkpoint_callback.on_epoch_end(epoch, {'loss': train_loss})
    
    print(f"Epoch {epoch+1}, Training Loss: {train_loss.numpy():.4f}")
    
    ## EarlyStopping 조건 충족 시 학습 중지
    #if early_stopping_callback.stopped_epoch is not None:
    #    print("학습 조기 종료")
    #    break

AttributeError: 'EarlyStopping' object has no attribute 'best'

In [ ]:
# 학습이 완료되면 테스트 데이터에 대한 예측값을 가져옵니다.
predicted_values = dann_model.predict(testx_n_elec)

In [ ]:
predicted_values.shape

In [ ]:
# Plot the predicted values against the actual values
plt.figure(figsize=(10, 6))
plt.plot(predicted_values, color='blue', label='Predicted')
plt.plot(testy_n_elec, color='red', linestyle='--', label='Actual')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs. Predicted Values')
plt.legend()
plt.show()